In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 1.自注意力
自注意力指的是 $Q,K,V$ 都是同一个序列$x_1,\dots,x_n,\forall x_i \in \mathbb{R}^d$,并通过自注意力池化层对序列抽取特征得到**长度相同**的序列 $y_1,\dots,y_n$，此处
$$y_i = f(x_i,(x_1,x_1),\dots,(x_n,x_n)) \in \mathbb{R}^d$$

In [2]:
num_hiddens, num_heads = 100, 5
attention = d2l.MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                                   num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [3]:
batch_size, num_queries, valid_lens = 2, 4, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
attention(X, X, X, valid_lens).shape

torch.Size([2, 4, 100])

## 2. 比较 Self-Attention 与 CNN、RNN
> 目标都是将由 $n$ 个词元组成的序列映射到另一个长度相等的序列
> 比较的是**计算复杂性**、**顺序操作**和**最大路径长度**

RNN 特别擅长记忆时序信息，但文本序列太长效果就差。
- Self-Attention 能无视距离，注意到从头到尾的全部序列。同时所需参数较少、处理并行度高的特点更加强了这一优点。但计算复杂度是$O(n^2 \cdot d)$ 因此**在很长的序列中计算会非常慢**。
- 与 CNN/RNN 不同，Self-Attention 不记录位置信息。如果将输入数据$x_i$ suffle 后再放到 Self-Attention 模型中，$y_i$的顺序可能变化，但值还是那些值。因此，处理时序数据时，不能纯用 Self-Attention，要使用 位置编码 $P$ 将位置信息注入到输入里。



## 位置编码 $P$
假设长度为 $n$ 的序列是$X \in \mathbb{R}^{n \times d}$,那么使用位置编码矩阵 $P \in \mathbb{R}^{n \times d}$ ,采用$X+P$ 作为自编码输入。位置编码可以学习也可以固定生成.

$P$在每一个(i,j)有一个不同的值，对位叠加到 $X$ 上。奇数列和偶数列计算公式是不同的三角函数（关键是具有周期性）。

### 相对位置信息
因为 $P$ 的计算公式是两个具有周期性的函数，那么 位于 $i + δ$ 行的位置编码可以通过线性变换投影到 $i$ 行

## QA
1. 自注意机制是不是也可以理解为一个网络层，就像CNN或RNN的一层来理解。
> 可以.能够被看作一个 layer

2. 位置编码在经过attention之后会丢失相对位置信息吗？
> 位置编码在 attention 后会经历损失,但不要紧,在输出时多多少少带一点就行